# Import

In [1]:
import os
import json
import pandas as pd
from collections import defaultdict
from cobra import io
from cobra.flux_analysis import flux_variability_analysis
from cobra.medium import minimal_medium

ImportError: cannot import name 'minimal_medium' from 'cobra.medium.minimal_medium' (/home/arog/miniconda3/envs/micom/lib/python3.10/site-packages/cobra/medium/minimal_medium.py)

# External Reaction Analysis

In [ ]:
# === CONFIGURATION ===
input_dir = "/home/arog/models/xml"               # Input directory
output_path = "/home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/nonzero_exchange_lbs.json"  # Output JSON

# === Store results in defaultdict(list)
nonzero_lbs = defaultdict(list)

# === Process all .xml files ===
for file_name in os.listdir(input_dir):
    if file_name.endswith(".xml"):
        model_path = os.path.join(input_dir, file_name)
        model_name = os.path.splitext(file_name)[0]
        print(f"Processing: {model_path}")

        try:
            model = io.read_sbml_model(model_path)
            
            for rxn in model.exchanges:
                if rxn.lower_bound != 0:
                    nonzero_lbs[rxn.id].append((model_name, rxn.lower_bound))
        
        except Exception as e:
            print(f"Failed to process {file_name}: {e}")

# === Convert to regular dict and sort by reaction ID ===
sorted_lbs = dict(sorted(nonzero_lbs.items()))

# === Save to JSON ===
with open(output_path, "w") as f:
    json.dump(sorted_lbs, f, indent=2)

print(f"\nSaved non-zero lower bounds to: {output_path}")


Processing: /home/arog/models/xml/iYL1228.xml
Processing: /home/arog/models/xml/iSDY_1059.xml
Processing: /home/arog/models/xml/iCN900.xml
Processing: /home/arog/models/xml/iECSE_1348.xml
Processing: /home/arog/models/xml/iUMNK88_1353.xml
Processing: /home/arog/models/xml/iETEC_1333.xml
Processing: /home/arog/models/xml/iECSF_1327.xml
Processing: /home/arog/models/xml/iEcSMS35_1347.xml
Processing: /home/arog/models/xml/iEC1356_Bl21DE3.xml
Processing: /home/arog/models/xml/iECUMN_1333.xml
Processing: /home/arog/models/xml/iNRG857_1313.xml
Processing: /home/arog/models/xml/iUTI89_1310.xml
Processing: /home/arog/models/xml/ic_1306.xml

Saved non-zero lower bounds to: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/nonzero_exchange_lbs.json


In [ ]:
# === CONFIGURATION ===            
output_path = "/home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/all_exchange.json"

# === Store results
r_ex_reactions = defaultdict(list)

# === Process all .xml files ===
for file_name in os.listdir(input_dir):
    if file_name.endswith(".xml"):
        model_path = os.path.join(input_dir, file_name)
        model_name = os.path.splitext(file_name)[0]
        print(f"Processing: {model_path}")

        try:
            model = io.read_sbml_model(model_path)

            for rxn in model.exchanges:  # model.exchanges gives reaction objects
                r_ex_reactions[rxn.id].append((model_name, rxn.lower_bound))

        except Exception as e:
            print(f"Failed to process {file_name}: {e}")

# === Sort and save ===
sorted_r_ex = dict(sorted(r_ex_reactions.items()))

with open(output_path, "w") as f:
    json.dump(sorted_r_ex, f, indent=2)

print(f"\nSaved R_EX reactions to: {output_path}")


Processing: /home/arog/models/xml/iYL1228.xml
Processing: /home/arog/models/xml/iSDY_1059.xml
Processing: /home/arog/models/xml/iCN900.xml
Processing: /home/arog/models/xml/iECSE_1348.xml
Processing: /home/arog/models/xml/iUMNK88_1353.xml
Processing: /home/arog/models/xml/iETEC_1333.xml
Processing: /home/arog/models/xml/iECSF_1327.xml
Processing: /home/arog/models/xml/iEcSMS35_1347.xml
Processing: /home/arog/models/xml/iEC1356_Bl21DE3.xml
Processing: /home/arog/models/xml/iECUMN_1333.xml
Processing: /home/arog/models/xml/iNRG857_1313.xml
Processing: /home/arog/models/xml/iUTI89_1310.xml
Processing: /home/arog/models/xml/ic_1306.xml

Saved R_EX reactions to: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/all_exchange.json


# Changing Bounds For iCN900:

In [ ]:
def update_model_bonds(model, json_path):
    """
    Updates the lower bounds of reactions in a community model based on a JSON file.

    Parameters:
        model (micom.Community or cobra.Model): The loaded community model.
        json_path (str): Path to the JSON file with reaction constraints.

    JSON Format:
    {
        "R_EX_glc__D_e": [["strain1", -5.0], ["strain2", -3.0]],
        ...
    }
    """
    with open(json_path, 'r') as f:
        reaction_bounds = json.load(f)

    updated = 0
    not_found = []

    for rxn_id, strain_flux_list in reaction_bounds.items():
        _ , new_lb = strain_flux_list[0]
        full_id = f"{rxn_id}"
        rxn = model.reactions.get_by_id(full_id) if full_id in model.reactions else None
        if rxn:
            rxn.lower_bound = new_lb
            updated += 1
        else:
            not_found.append(full_id)

    print(f"Updated {updated} reactions.")
    if not_found:
        print(f"Reactions not found in model ({len(not_found)}):")
        for r in not_found:
            print(f"  - {r}")

    return model

In [ ]:
model_path = "/home/arog/models/xml/iCN900.xml"
json_path = "/home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/nonzero_exchange_lbs.json"

model = io.read_sbml_model(model_path)
model = update_model_bonds(model, json_path)
io.write_sbml_model(model, filename = model_path)

✅ Updated 26 reactions.
⚠️ Reactions not found in model (10):
  - EX_cbl1_e
  - EX_cl_e
  - EX_cu2_e
  - EX_fe3_e
  - EX_mn2_e
  - EX_mobd_e
  - EX_ni2_e
  - EX_o2_e
  - EX_so4_e
  - EX_tungs_e


# Individual Model Simulations

In [ ]:
# === CONFIGURATION ===
input_dir = "/home/arog/models/xml"            # Directory containing .xml files
output_dir = "/home/arog/Documents/GitHub/HGMB_Project/solo_sim_results"   # Directory to save results
fraction_of_optimum = 0.9                      # FVA at 100% of optimal growth

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)

# === Process all .xml files ===
for file_name in os.listdir(input_dir):
    if file_name.endswith(".xml"):
        model_path = os.path.join(input_dir, file_name)
        print(f"Processing: {model_path}")
        
        try:
            # === Load model ===
            model = io.read_sbml_model(model_path)
            base_name = os.path.splitext(file_name)[0]

            # === Save model summary ===
            summary_text = str(model.summary())
            summary_path = os.path.join(output_dir, f"{base_name}_summary.txt")
            with open(summary_path, "w") as f:
                f.write(summary_text)
            print(f"Saved summary: {summary_path}")

            # === Get and save minimal medium ===
            min_med = minimal_medium(model, open_exchanges = True)
            minmed_path = os.path.join(output_dir, f"{base_name}_minimal_medium.csv")
            min_med.to_csv(minmed_path)
            print(f"Saved minimal medium: {minmed_path}")

            # === Run and save FVA ===
            fva_result = flux_variability_analysis(model, fraction_of_optimum=fraction_of_optimum)
            fva_path = os.path.join(output_dir, f"{base_name}_fva.csv")
            fva_result.to_csv(fva_path)
            print(f"Saved FVA: {fva_path}")



        except Exception as e:
            print(f"Failed to process {file_name}: {e}")


Processing: /home/arog/models/xml/iYL1228.xml
Saved summary: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iYL1228_summary.txt
Saved minimal medium: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iYL1228_minimal_medium.csv
Saved FVA: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iYL1228_fva.csv
Processing: /home/arog/models/xml/iSDY_1059.xml
Saved summary: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iSDY_1059_summary.txt
Saved minimal medium: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iSDY_1059_minimal_medium.csv
Saved FVA: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iSDY_1059_fva.csv
Processing: /home/arog/models/xml/iCN900.xml
Saved summary: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iCN900_summary.txt
Saved minimal medium: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iCN900_minimal_medium.csv
Saved FVA: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iCN900_fva.csv
Pro

# Individual Model Simulations - Gut Medium

In [ ]:
gut_medium_dict = {
    # Carbohydrates & SCFAs
    "R_EX_glc__D_e": (-10.0, 5.0),
    "R_EX_ac_e": (-5.0, 5.0),
    "R_EX_but_e": (-5.0, 5.0),
    "R_EX_lac__D_e": (-2.0, 5.0),
    "R_EX_lac__L_e": (-2.0, 5.0),
    "R_EX_succ_e": (-2.0, 5.0),

    # Amino acids
    "R_EX_ala__L_e": (-1.0, 5.0),
    "R_EX_arg__L_e": (-1.0, 5.0),
    "R_EX_asn__L_e": (-1.0, 5.0),
    "R_EX_asp__L_e": (-1.0, 5.0),
    "R_EX_cys__L_e": (-1.0, 5.0),
    "R_EX_gln__L_e": (-1.0, 5.0),
    "R_EX_glu__L_e": (-1.0, 5.0),
    "R_EX_gly_e": (-1.0, 5.0),
    "R_EX_his__L_e": (-1.0, 5.0),
    "R_EX_ile__L_e": (-1.0, 5.0),
    "R_EX_leu__L_e": (-1.0, 5.0),
    "R_EX_lys__L_e": (-1.0, 5.0),
    "R_EX_met__L_e": (-1.0, 5.0),
    "R_EX_phe__L_e": (-1.0, 5.0),
    "R_EX_pro__L_e": (-1.0, 5.0),
    "R_EX_ser__L_e": (-1.0, 5.0),
    "R_EX_thr__L_e": (-1.0, 5.0),
    "R_EX_trp__L_e": (-1.0, 5.0),
    "R_EX_tyr__L_e": (-1.0, 5.0),
    "R_EX_val__L_e": (-1.0, 5.0),

    # Inorganic ions
    "R_EX_ca2_e": (-0.1, 5.0),
    "R_EX_cl_e": (-0.1, 5.0),
    "R_EX_k_e": (-1.0, 5.0),
    "R_EX_mg2_e": (-0.1, 5.0),
    "R_EX_mn2_e": (-0.01, 5.0),
    "R_EX_nh4_e": (-5.0, 5.0),
    "R_EX_pi_e": (-2.0, 5.0),
    "R_EX_so4_e": (-0.5, 5.0),
    "R_EX_fe2_e": (-0.01, 5.0),
    "R_EX_zn2_e": (-0.005, 5.0),

    # Vitamins & cofactors
    "R_EX_nac_e": (-0.01, 5.0),
    "R_EX_thm_e": (-0.01, 5.0),
    "R_EX_btn_e": (-0.01, 5.0),
    "R_EX_pnto__R_e": (-0.01, 5.0),
    "R_EX_ribflv_e": (-0.01, 5.0),
    # Gases
    "R_EX_o2_e": (0.0, 0.0),           # strict anaerobic
    "R_EX_h2o_e": (-1000.0, 1000.0),
    "R_EX_co2_e": (-1000.0, 1000.0)
}

In [ ]:
len(gut_medium_dict)

44

In [ ]:
# === CONFIGURATION ===
input_dir = "/home/arog/models/xml"            # Directory containing .xml files
output_dir = "/home/arog/Documents/GitHub/HGMB_Project/solo_sim_results"   # Directory to save results
fraction_of_optimum = 0.9                      # FVA at 100% of optimal growth

# === Ensure output directory exists ===
os.makedirs(output_dir, exist_ok=True)
# === Process all .xml files ===
for file_name in os.listdir(input_dir):
    if file_name.endswith(".xml"):
        model_path = os.path.join(input_dir, file_name)
        print(f"Processing: {model_path}")

        try:
            # === Load model ===
            model = io.read_sbml_model(model_path)
            base_name = os.path.splitext(file_name)[0]

            # === Apply gut medium restrictions ===
            for rxn_id, (lb, ub) in gut_medium_dict.items():
                if rxn_id[2:] in model.exchanges:
                    rxn = model.exchanges.get_by_id(rxn_id[2:])
                    rxn.lower_bound = lb
                    rxn.upper_bound = ub

            # === Save model summary ===
            summary_text = str(model.summary())
            summary_path = os.path.join(output_dir, f"{base_name}_summary_gut.txt")
            with open(summary_path, "w") as f:
                f.write(summary_text)
            print(f"Saved summary: {summary_path}")

            # === Get and save minimal medium ===
            min_med = minimal_medium(model, open_exchanges=False)
            minmed_path = os.path.join(output_dir, f"{base_name}_minimal_medium_gut.csv")
            min_med.to_csv(minmed_path)
            print(f"Saved minimal medium: {minmed_path}")

            # === Run and save FVA ===
            fva_result = flux_variability_analysis(model, fraction_of_optimum=fraction_of_optimum)
            fva_path = os.path.join(output_dir, f"{base_name}_fva_gut.csv")
            fva_result.to_csv(fva_path)
            print(f"Saved FVA: {fva_path}")

        except Exception as e:
            print(f"❌ Failed to process {file_name}: {e}")

Processing: /home/arog/models/xml/iYL1228.xml
Saved summary: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iYL1228_summary_gut.txt
Saved minimal medium: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iYL1228_minimal_medium_gut.csv
Saved FVA: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iYL1228_fva_gut.csv
Processing: /home/arog/models/xml/iSDY_1059.xml
Saved summary: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iSDY_1059_summary_gut.txt
Saved minimal medium: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iSDY_1059_minimal_medium_gut.csv
Saved FVA: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iSDY_1059_fva_gut.csv
Processing: /home/arog/models/xml/iCN900.xml


Minimization of medium was infeasible.


Saved summary: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iCN900_summary_gut.txt
❌ Failed to process iCN900.xml: 'NoneType' object has no attribute 'to_csv'
Processing: /home/arog/models/xml/iECSE_1348.xml
Saved summary: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iECSE_1348_summary_gut.txt
Saved minimal medium: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iECSE_1348_minimal_medium_gut.csv
Saved FVA: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iECSE_1348_fva_gut.csv
Processing: /home/arog/models/xml/iUMNK88_1353.xml
Saved summary: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iUMNK88_1353_summary_gut.txt
Saved minimal medium: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iUMNK88_1353_minimal_medium_gut.csv
Saved FVA: /home/arog/Documents/GitHub/HGMB_Project/solo_sim_results/iUMNK88_1353_fva_gut.csv
Processing: /home/arog/models/xml/iETEC_1333.xml
Saved summary: /home/arog/Documents/GitHub/HGMB_Project/solo_